# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# Read in file
weather_df = pd.read_csv("../output_data/cities_export.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Leku,6.8667,38.4500,287.32,51,14,2.51,ET,1611964345
1,1,Butaritari,3.0707,172.7902,300.27,75,95,7.36,KI,1611964345
2,2,Ucluelet,48.9329,-125.5528,278.71,93,90,6.69,CA,1611964345
3,3,Atuona,-9.8000,-139.0333,299.11,76,5,8.77,PF,1611964318
4,4,Kaitangata,-46.2817,169.8464,286.48,75,80,3.58,NZ,1611964346


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weather_df['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Created hotel_df to hold rows that meet ideal conditions
hotel_df = weather_df.loc[(weather_df['Max Temp']>293)  & (weather_df['Max Temp']< 302) & (weather_df['Wind Speed']<10) & (weather_df['Cloudiness'] < 40)]
hotel_df.shape

(60, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
# hotel_df["Hotel Name"] = "placeholder"
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Atuona,-9.8000,-139.0333,299.11,76,5,8.77,PF,1611964318
16,16,Kédougou,12.5579,-12.1743,296.55,18,4,0.90,SN,1611964349
22,22,Saint-Philippe,-21.3585,55.7679,298.15,65,0,3.09,RE,1611964350
30,30,Hobart,-42.8794,147.3294,297.04,53,20,3.60,AU,1611964283
38,38,Guaratinga,-16.5667,-39.5667,299.15,69,20,5.66,BR,1611964355


In [17]:
# Code to build path for hotel name
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()

params = {
    "location": "39.952583,-75.16522", 
    "type": "lodging",
    "key": g_key,
}
# results = response['results']
pprint(response)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -39.54732789999999,
                                        'lng': 174.132167},
                           'viewport': {'northeast': {'lat': -39.5460101197085,
                                                      'lng': 174.1335104802915},
                                        'southwest': {'lat': -39.5487080802915,
                                                      'lng': 174.1308125197085}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/camping-71.png',
              'name': 'Freedom Campground',
              'photos': [{'height': 2268,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/114347178502595642474">René '
                                                'Redanz</a>'],
                          'photo_reference': 'ATtY

In [18]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']
    city= row['City']
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except (KeyError, IndexError):
        print(f"Missing info for {city}.")
        
    print("------------")

Closest Atuona hotel is Villa Enata.
------------


/Users/drewmcbride/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/drewmcbride/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Closest Kédougou hotel is Relais de Kédougou.
------------
Closest Saint-Philippe hotel is Chambres d'hôte "La Trinité".
------------
Closest Hobart hotel is St Ives Hobart Accommodation Tasmania.
------------
Closest Guaratinga hotel is Hotel Caraíva.
------------
Closest Touros hotel is INN NEW HORIZON.
------------
Closest Pitimbu hotel is Reserva do Abiaí.
------------
Closest Road Town hotel is ZINGARA 76ft Catamaran, full crew-all included.
------------
Closest Pisco hotel is Hotel San Isidro Oficial.
------------
Closest Mar del Plata hotel is Gran Hotel Mar del Plata.
------------
Closest Grand Gaube hotel is Veranda Paul et Virginie Hotel & Spa.
------------
Closest Santa Maria hotel is Hotel Morotin.
------------
Closest East London hotel is Tu Casa.
------------
Closest Carnarvon hotel is Hospitality Carnarvon.
------------
Closest Wau hotel is Universal Gab Hotel.
------------
Closest Port-Gentil hotel is Fengshui Residence.
------------
Closest George Town hotel is Cititel

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Formatting figure

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…